<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Wyjanienie-modelu---analiza-bdw/Similarity_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1829, done.
remote: Counting objects: 100% (1829/1829), done.
remote: Compressing objects: 100% (804/804), done.
remote: Total 1829 (delta 1135), reused 1657 (delta 1006), pack-reused 0
Receiving objects: 100% (1829/1829), 6.08 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (1135/1135), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!git checkout Wyjanienie-modelu---analiza-bdw

Branch 'Wyjanienie-modelu---analiza-bdw' set up to track remote branch 'Wyjanienie-modelu---analiza-bdw' from 'origin'.
Switched to a new branch 'Wyjanienie-modelu---analiza-bdw'


In [4]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=65b40ff9eda5c0c1cb525e377c4f5570863989f5bd159ef25ac00d314e8f6643
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-22 16:11:46--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [12]:
import pandas as pd
from lib.pyspark_init import create_spark_context
from lib.similarity_utils import *
from lib.const import TRAIN_DATA, VALIDATE_DATA, SIMILARITY_VALUES, RANKINGS

In [6]:
spark = create_spark_context()
spark

In [7]:
!chmod +x download_data.sh
!./download_data.sh

--2023-01-22 16:11:58--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 65.9.86.88, 65.9.86.16, 65.9.86.82, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|65.9.86.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239771823 (229M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 228.66M  87.5MB/s    in 2.6s    

2023-01-22 16:12:01 (87.5 MB/s) - ‘name.basics.tsv.gz’ saved [239771823/239771823]

--2023-01-22 16:12:01--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 65.9.86.88, 65.9.86.16, 65.9.86.82, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|65.9.86.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294377872 (281M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 280.74M  76.2MB/s    in 3.7s  

In [8]:
%%time
train_data = pd.read_parquet(TRAIN_DATA, columns=["nconst", "tconst", "titleType", "genres", "category", "primaryName", "knownForTitles", "average_films_rating_norm", "genres_code", "types_code"])
validate_data = pd.read_parquet(VALIDATE_DATA, columns=["nconst", "tconst", "titleType", "genres", "category", "primaryName", "knownForTitles", "average_films_rating_norm", "genres_code", "types_code"])

CPU times: user 29.8 s, sys: 4.87 s, total: 34.7 s
Wall time: 43.3 s


# Porównywanie dwóch aktorów

Wyciągnięcie pierwszego aktora

In [9]:
actor1 = "Natalie Portman"

In [30]:
if (train_data['primaryName'] == actor1).any():
    actor1_data = find_actor(train_data,  train_data[train_data['primaryName'] == actor1].iloc[0]['nconst'])
    display(pd.DataFrame(actor1_data))
    actor1_data = prepare_pandas_row(actor1_data)
elif (validate_data['primaryName'] == actor1).any():
    actor1_data = find_actor(validate_data,  validate_data[validate_data['primaryName'] == actor1].iloc[0]['nconst'])
    display(pd.DataFrame(actor1_data))
    actor1_data = prepare_pandas_row(actor1_data)
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
titleType,"[movie, movie, video, short, short, movie, mov..."
genres,"[Action, Adventure, Fantasy, Drama, Short, Dra..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
average_films_rating_norm,0.666
genres_code,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, ..."
types_code,"[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0]"


Wyciągnięcie drugiego aktora

In [25]:
actor2 = "Keira Knightley"

In [31]:
if (train_data['primaryName'] == actor2).any():
    actor2_data = find_actor(train_data,  train_data[train_data['primaryName'] == actor2].iloc[0]['nconst'])
    display(pd.DataFrame(actor2_data))
    actor2_data = prepare_pandas_row(actor2_data)
elif (validate_data['primaryName'] == actor2).any():
    actor2_data = find_actor(validate_data,  validate_data[validate_data['primaryName'] == actor2].iloc[0]['nconst'])
    display(pd.DataFrame(actor2_data))
    actor2_data = prepare_pandas_row(actor2_data)
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,993366
nconst,nm0461136
tconst,"[tt0368925, tt0366627, tt0409997, tt1307068, t..."
titleType,"[videoGame, movie, video, movie, movie, movie,..."
genres,"[Adventure, Fantasy, Drama, Fantasy, Mystery, ..."
category,actress
primaryName,Keira Knightley
knownForTitles,"[tt0325980, tt2084970, tt0783233, tt0414387]"
average_films_rating_norm,0.648
genres_code,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, ..."
types_code,"[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0]"


Wyniki similarity dla tych dwóch aktorów

In [33]:
similarity_value = similarity(actor1_data, actor2_data, True)
print("Wartość similarity dla ", actor1, " i ", actor2, " wynosi: ", similarity_value)

Wartość similarity dla  Natalie Portman  i  Keira Knightley  wynosi:  0.16222222222222227


Zapisanie porównywanych par aktorów wraz wartością similarity do pliku na Google Drive.

In [37]:
file_object = open(SIMILARITY_VALUES, 'a')
text = "{}, {}, similarity value: {}\n".format(actor1, actor2, similarity_value)
file_object.write(text)
file_object.close()

Zapisanie wniosków, jeśli wyszło coś ciekawego

# Ranking najbardziej podobnych aktoró do podanego

Wyciągnięcie aktora

In [38]:
actor = "Natalie Portman"

In [43]:
if (train_data['primaryName'] == actor).any():
    actor_data = find_actor(train_data,  train_data[train_data['primaryName'] == actor].iloc[0]['nconst'])
    display(pd.DataFrame(actor_data))
    actor_id = actor_data['nconst']
elif (validate_data['primaryName'] == actor).any():
    actor_data = find_actor(validate_data,  validate_data[validate_data['primaryName'] == actor].iloc[0]['nconst'])
    display(pd.DataFrame(actor_data))
    actor_id = actor_data['nconst']
else:
    print("Aktora nie ma w danych treningowych i walidacyjnych. Wybierz innego aktora.")

,337799
nconst,nm0000204
tconst,"[tt0121766, tt1032825, tt8918080, tt1094249, t..."
titleType,"[movie, movie, video, short, short, movie, mov..."
genres,"[Action, Adventure, Fantasy, Drama, Short, Dra..."
category,actress
primaryName,Natalie Portman
knownForTitles,"[tt0947798, tt0376541, tt0110413, tt0434409]"
average_films_rating_norm,0.666
genres_code,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, ..."
types_code,"[0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0]"


Ranking 3 najbardziej podobnych aktorów do podanego powyżej

In [46]:
if (train_data['primaryName'] == actor).any():
    ranking = get_ranking(train_data, actor_id, 3, True)
    ranking_names = replace_ids_with_names(train_data, ranking, True)
elif (validate_data['primaryName'] == actor).any():
    ranking = get_ranking(validate_data, actor_id, 3, True)
    ranking_names = replace_ids_with_names(validate_data, ranking, True)
print(ranking_names)

TypeError: ignored

Zapisanie do pliku

In [ ]:
file_object = open(RANKINGS, 'a')
text = "{}, Najbardziej podobni: {}\n".format(actor, ranking_names)
file_object.write(text)
file_object.close()